# How to build a simple Agent LLM App with LangGraph
* Very basic tool-using Agent LLM App with memory.

Language models can't do anything on their own; they can only create text based on what you ask them. However, LangChain allows people to build agents—think of these as smart systems or helpers— that use language models to think and decide what to do next.

Here’s how it works:
1. **Use the Language Model as a Brain**: The agent uses the language model to figure out which actions it should take, based on the information it has or what you ask it to do.
2. **Taking Action**: After deciding, the agent then goes ahead and does those actions.
3. **Learning and Adjusting**: Once the actions are done, the results can be given back to the language model. This helps the model check if everything is complete or if it needs to do something else.

So, essentially, LangChain helps turn a language model from just a tool for writing and answering into a system that can act and react, almost like a very simple robot brain.

Here we will build an agent that can interact with a search engine. You will be able to ask this agent questions, watch it calling the search tool, and have conversations with it.

## Concepts included
* [LangGraph](https://python.langchain.com/v0.2/docs/concepts/#langgraph).

LangGraph is a library created by the LangChain team for building stateful, multi-actor applications with LLMs, used to create agent and multi-agent workflows.

## Setup

#### After you download the code from the github repository in your computer
In terminal:
* cd project_name
* pyenv local 3.11.4
* poetry install
* poetry shell

#### To open the notebook with Jupyter Notebooks
In terminal:
* jupyter lab

Go to the folder of notebooks and open the right notebook.

#### To see the code in Virtual Studio Code or your editor of choice.
* open Virtual Studio Code or your editor of choice.
* open the project-folder
* open the 001-simple-agent.py file

## Create your .env file
* In the github repo we have included a file named .env.example
* Rename that file to .env file and here is where you will add your confidential api keys. Remember to include:
* OPENAI_API_KEY=your_openai_api_key
* LANGCHAIN_TRACING_V2=true
* LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
* LANGCHAIN_API_KEY=your_langchain_api_key
* LANGCHAIN_PROJECT=your_project_name

We will call our LangSmith project **001-simple-agent**.

## Connect with the .env file located in the same directory of this notebook

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [40]:
#!pip install python-dotenv

In [41]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

#### Install LangChain

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [42]:
#!pip install langchain

## Connect with an LLM and start a conversation with it

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [43]:
#!pip install langchain-openai

* For this project, we will use OpenAI's gpt-3.5-turbo

In [44]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

#### Track the operation in LangSmith
* [Open LangSmith here](smith.langchain.com)

## Agents
* Agents use LLMs as reasoning engines to determine which actions to take.

## Tool-using Agent
* For this basic agent we will use just one tool. In next advanced projects, you will learn how to use agents with several tools.
* **Our tool of choice will be Tavily** - a search engine. 
#### Tavily API Key
* You will need to add your Tavily API key in the .env file. Obtain a key by signing up on their [website](https://tavily.com/).

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [45]:
#pip install langgraph

#### Define tools

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [46]:
#pip install langchain-community

In [47]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults(max_results=2)
search.invoke("Who are the top stars of the 2024 Eurocup?")

[{'title': 'Five of the most impressive performers from EURO 2024',
  'url': 'https://www.flashscore.com/news/soccer-euro-five-of-the-most-impressive-performers-from-euro-2024/KxbVz2L8/',
  'content': "Here, AFP Sport looks at five stars who have broken through with impressive performances at Euro 2024: Dani Olmo (Spain) Spanish starlets Yamal and Nico Williams have lit up Euro 2024 from the flanks but Dani Olmo's contributions from the middle of the park have proved crucial for the finalists. The 26-year-old is well-known in Germany after four years at RB Leipzig but has made everyone else",
  'score': 0.81489426},
 {'title': 'Rodri, Musiala, Yamal, Pepe: Top 10 players from UEFA Euro 2024',
  'url': 'https://www.olympics.com/en/news/rodri-musiala-pepe-top-10-players-from-uefa-euro-2024',
  'content': 'Euro 2024: Top 10 players from the competition · Rodri (Spain) · Lamine Yamal (Spain) · Kobbie Mainoo (England) · Nico Williams (Spain) · Jamal',
  'score': 0.6227582}]

In [48]:
tools = [search]

* **In order to enable this model to do tool calling we use .bind_tools** to give the language model knowledge of these tools:

In [49]:
llm_with_tools = llm.bind_tools(tools)

## Create the agent
* We will be using LangGraph to construct the agent. 
* **Note that below we are passing in the origina chat model, not the llm_with_tools we built later**. That is because create_tool_calling_executor will call .bind_tools for us under the hood.

In [50]:
# from langgraph.prebuilt import chat_agent_executor

# agent_executor = chat_agent_executor.create_tool_calling_executor(llm, tools)

* In the latest version of langgraph, we have a most modern way to do this:

In [51]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools)

## Run the agent
* Let's first try it with .invoke():

In [52]:
from langchain_core.messages import HumanMessage

response = agent_executor.invoke({"messages": [HumanMessage(content="Where is the soccer Eurocup 2024 played?")]})

response["messages"]

[HumanMessage(content='Where is the soccer Eurocup 2024 played?', additional_kwargs={}, response_metadata={}, id='002e6e5f-d4f4-4f6b-96fe-35b0a094c518'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_tCRE8DCWrcHeeIEUh5Ql7Yyx', 'function': {'arguments': '{"query":"Eurocup 2024 location"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 90, 'total_tokens': 113, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BnAioxyaq4LTMQaQvMu5T6t1Bx3LG', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--c41fa309-4e9f-4adf-b25a-43d99c8a435e-0', tool_calls=[{'name': 'tavily_search_results_j

* Now let's try it with .stream():

In [53]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Who was Cleopatra?")]}):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="Cleopatra VII Philopator (69–30 BC) was the last active ruler of the Ptolemaic Kingdom of Egypt. She is one of the most famous female figures in history, known for her intelligence, political acumen, and romantic liaisons with powerful Roman leaders such as Julius Caesar and Mark Antony. \n\nCleopatra was born in Alexandria, Egypt, and was a member of the Ptolemaic dynasty, which was of Macedonian Greek origin. She ascended to the throne in 51 BC and initially ruled alongside her brother Ptolemy XIII. Cleopatra is often remembered for her efforts to revive Egypt's fortunes amidst political turmoil and her skillful alliances with Rome, which ultimately led to her downfall.\n\nHer reign came to an end after the defeat of her forces alongside Mark Antony at the Battle of Actium in 31 BC. Following this defeat, Cleopatra and Antony both committed suicide, marking the end of Ptolemaic rule and the annexation of Egypt into the Roman Empire.\n\nCleop

## Adding memory
* Adding memory in LangGraph is very similar to what we did with LangChain.

In [54]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

* Let's create our new agent with memory and set one thread_id so the agent can create a memory for each session as we did with our previous conversational RAG app:

In [55]:
agent_executor = create_react_agent(llm, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "001"}}

* Let's now try this new agent with .stream():

In [56]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Who is Buffy the Vampyre Slaywer?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='"Buffy the Vampire Slayer" is a popular television series created by Joss Whedon that premiered in 1997. The show follows the story of Buffy Summers, a young woman chosen to battle against vampires, demons, and other supernatural foes. Buffy is portrayed by actress Sarah Michelle Gellar.\n\nThe series blends elements of horror, drama, and comedy while addressing themes such as friendship, love, and self-discovery. Buffy is known for her strength, resilience, and cleverness, often facing difficult challenges while trying to maintain a normal life as a high school and later college student.\n\nThe character and the show have had a significant cultural impact, leading to several spin-offs, comic books, and a devoted fan base. The series ran for seven seasons and has been praised for its character development and modern take on the horror genre.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 169, 'pr

In [57]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Who will play the slayer in the reboot, and will Spike be there?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_tGPmoh0p6p8V4tqn7WUOwFMp', 'function': {'arguments': '{"query":"Buffy the Vampire Slayer reboot cast news 2023"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 281, 'total_tokens': 309, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BnAj1G2iJkXd8C8rQxfpvyAknRg9g', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--2d33f721-dbcf-4753-855d-bb4da712564c-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Buffy the Vampire Slayer reboot cast news 2023'}, 'id': 'call_tGPmoh0p6p8V4tqn7W

* Let's change the thread_id and see what happens:

In [58]:
config = {"configurable": {"thread_id": "002"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="About what tv show we were talking?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="Could you please provide a bit more detail or context about the TV show you're referring to? This will help me assist you better!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 87, 'total_tokens': 114, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BnAjF1b8fTSMwLXXn9wa1Fx4nIkl6', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--a55d3014-8ac4-467b-a99d-b18561f25f74-0', usage_metadata={'input_tokens': 87, 'output_tokens': 27, 'total_tokens': 114, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
----


* As you can see, when we changed the threat_id we started a different conversation memory, so our app does not remember the previous interaction and instead of giving us the right answer it decided to hallucinate.

## How to execute the code from Visual Studio Code
* In Visual Studio Code, see the file 001-simple-agent.py
* In terminal, make sure you are in the directory of the file and run:
    * python 001-simple-agent.py
        *
Not working